In [4]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [6]:
config = get_config(nb_name)

In [7]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [22]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [9]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [23]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [24]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [25]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                               VALID:        \
       Time   Ep   Ct     LOSS   PPL      NLL     KL   REG     LOSS   PPL   
5000     74    8  463  9533.87  1781  9511.69  22.00  0.19  9373.25  1750   
10000    65   17  360  9505.93  1742  9483.42  22.32  0.17  9363.04  1736   
15000    68   26  257  9493.16  1724  9470.73  22.24  0.16  9359.11  1729   
20000    68   35  154  9485.31  1714  9462.93  22.14  0.15  9357.38  1727   
25000    67   44   51  9479.77  1707  9457.43  22.03  0.15  9355.67  1724   
30000    71   52  515  9475.86  1701  9453.62  21.93  0.14  9354.64  1723   
35000    89   61  412  9473.04  1697  9450.82  21.84  0.14  9355.26  1723   
40000    78   70  309  9470.78  1694  9448.55  21.76  0.14  9355.38  1724   
45000    65   79  206  9468.79  1692  9446.53  21.69  0.13  9355.14  1724   
50000    64   88  103  9467.14  1689  9444.86  21.64  0.13  9354.53  1723   
55000    80   97    0  9465.68  1688  9443.37  21.59  0.13  9355.73  1725   
60000    76  105  464  9464.52  1686  9442.30  21.55  0.13  9355.58  1724   
65000    76  114  361  9463.55  1685  9441.48  21.52  0.13  9355.24  1724   
70000    76  123  258  9462.57  1683  9440.64  21.49  0.13  9355.59  1724   
75000    75  132  155  9461.66  1682  9439.85  21.46  0.12  9355.69  1724   
80000    79  141   52  9460.79  1681  9439.08  21.44  0.12  9355.31  1724   
85000    73  149  516  9460.06  1680  9438.47  21.42  0.12  9355.68  1725   
90000    69  158  413  9459.43  1680  9437.94  21.40  0.12  9355.70  1725   
95000    69  167  310  9458.87  1679  9437.47  21.38  0.12  9355.21  1724   
100000   73  176  207  9458.27  1678  9436.96  21.36  0.12  9355.90  1725   
105000   73  185  104  9457.73  1678  9436.50  21.35  0.12  9355.83  1725   
110000   76  194    1  9457.19  1677  9436.04  21.34  0.12  9355.51  1725   
115000   73  202  465  9456.75  1676  9435.67  21.32  0.12  9355.67  1725   
120000   74  211  362  9456.38  1676  9435.34  21.31  0.12  9355.53  1725   
125000   69  220  259  9455.99  1676  9435.00  21.30  0.11  9355.85  1725   
130000   65  229  156  9455.62  1675  9434.65  21.29  0.11  9355.77  1725   
135000   64  238   53  9455.22  1675  9434.30  21.28  0.11  9355.94  1725   
140000   85  246  517  9454.91  1674  9434.01  21.27  0.11  9355.92  1725   
145000   66  255  414  9454.63  1674  9433.75  21.26  0.11  9355.40  1725   
150000   66  264  311  9454.39  1674  9433.52  21.25  0.11  9355.56  1724   
155000   71  273  208  9454.11  1673  9433.26  21.24  0.11  9357.06  1727   
160000   76  282  105  9453.86  1673  9433.02  21.24  0.11  9355.88  1726   
165000   79  291    2  9453.57  1673  9432.74  21.23  0.11  9356.35  1726   
170000   76  299  466  9453.35  1672  9432.53  21.22  0.11  9355.69  1725   
175000   79  308  363  9453.17  1672  9432.36  21.21  0.11  9356.15  1726   
180000   74  317  260  9452.97  1672  9432.17  21.21  0.11  9356.17  1725   
185000   76  326  157  9452.76  1672  9431.97  21.20  0.11  9356.46  1726   
190000   78  335   54  9452.54  1671  9431.76  21.19  0.11  9356.28  1726   
195000   69  343  518  9452.36  1671  9431.58  21.19  0.11  9355.97  1726   
200000   70  352  415  9452.20  1671  9431.43  21.18  0.11  9356.37  1726   
205000   75  361  312  9452.07  1671  9431.30  21.18  0.10  9356.56  1726   
210000   77  370  209  9451.90  1670  9431.14  21.17  0.10  9357.64  1729   
215000   74  379  106  9451.75  1670  9431.00  21.17  0.10  9355.81  1725   

                                TEST:       SPEC:             HIER:        
            NLL     KL   REG     LOSS   PPL     1     2     3 CHILD OTHER  
5000    9350.33  22.76  0.15  9373.19  1750  0.37  0.34  0.42  0.38  0.31  
10000   9340.63  22.26  0.14  9363.26  1736  0.39  0.37  0.42  0.36  0.29  
15000   9336.83  22.15  0.14  9359.06  1729  0.39  0.39  0.43  0.34  0.28  
20000   9335.55  21.70  0.13  9356.75  1727  0.39  0.40  0.44  0.32  0.27  
25000   9334.09  21.46  0.13  9355.44  1724  0.40  0.40  0.43  0.33  0.28  
30000   9333.33  21.20  0.

0 R: 1.000 P: 0.105 speech user dialogue human utterance task time speaker figure utterances
   1 R: 0.305 P: 0.057 models neural embeddings training network vector task learning representation embedding
     11 R: 0.071 P: 0.071 translation english source target alignment phrase translations pairs sentences parallel
     12 R: 0.103 P: 0.103 training models probability learning feature performance algorithm number test method
     13 R: 0.073 P: 0.073 tree grammar rules node rule parsing trees nodes figure structure
   2 R: 0.266 P: 0.065 sentiment classification feature tweets classifier task positive negative analysis training
     21 R: 0.069 P: 0.069 languages english morphological character segmentation errors chinese error dictionary characters
     22 R: 0.073 P: 0.073 document topic documents text similarity sentences terms method number term
     23 R: 0.059 P: 0.059 annotation dependency pos parser syntactic sentences tags parsing annotated treebank
   3 R: 0.324 P: 0.070 en

KeyboardInterrupt: 

In [31]:
idx_to_word[bow_idxs[0]]

'training'